<a href="https://colab.research.google.com/github/Zerzavot/GlobalAIHubDLCourse/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("Loan_Data.csv", index_col = 0)
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [ ]:
df.shape

(614, 12)

In [ ]:
df.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [ ]:
df.Education.unique()

array(['Graduate', 'Not Graduate'], dtype=object)

In [ ]:
df.Dependents = df.Dependents.replace({"0":0,"1":1,"2":2, "3+":3})
df.Dependents.unique()

array([ 0.,  1.,  2.,  3., nan])

In [ ]:
df.isnull().mean()

Gender               0.021173
Married              0.004886
Dependents           0.024430
Education            0.000000
Self_Employed        0.052117
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           0.035831
Loan_Amount_Term     0.022801
Credit_History       0.081433
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64

In [ ]:
df.dropna(inplace = True)
df.isnull().mean()


Gender               0.0
Married              0.0
Dependents           0.0
Education            0.0
Self_Employed        0.0
ApplicantIncome      0.0
CoapplicantIncome    0.0
LoanAmount           0.0
Loan_Amount_Term     0.0
Credit_History       0.0
Property_Area        0.0
Loan_Status          0.0
dtype: float64

In [ ]:
df.shape

(480, 12)

In [ ]:
X = df.drop("Loan_Status", axis =  1)
y = df["Loan_Status"]
X = pd.get_dummies(X, drop_first = True)
X.dtypes

Dependents                 float64
ApplicantIncome              int64
CoapplicantIncome          float64
LoanAmount                 float64
Loan_Amount_Term           float64
Credit_History             float64
Gender_Male                  uint8
Married_Yes                  uint8
Education_Not Graduate       uint8
Self_Employed_Yes            uint8
Property_Area_Semiurban      uint8
Property_Area_Urban          uint8
dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

mu = X_train.mean()
sigma = X_train.std()

X_train = (X_train - mu)/ sigma
X_test = (X_test - mu)/ sigma

In [ ]:
from sklearn.utils import class_weight
from sklearn.linear_model import LogisticRegression

mdl = LogisticRegression(penalty = "none",class_weight={"N":1, "Y":1})

mdl.fit(X_train, y_train)
ypred_train = mdl.predict(X_train)
ypred_test = mdl.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_train, ypred_train))
print(classification_report(y_test, ypred_test))

              precision    recall  f1-score   support

           N       0.88      0.46      0.60       113
           Y       0.81      0.97      0.89       271

    accuracy                           0.82       384
   macro avg       0.85      0.72      0.75       384
weighted avg       0.83      0.82      0.80       384

              precision    recall  f1-score   support

           N       0.92      0.34      0.50        35
           Y       0.72      0.98      0.83        61

    accuracy                           0.75        96
   macro avg       0.82      0.66      0.67        96
weighted avg       0.80      0.75      0.71        96



In [ ]:
mdl = LogisticRegression(C = 0.1,class_weight={"N":1.5, "Y":1}, solver = "newton-cg")

mdl.fit(X_train, y_train)
ypred_train = mdl.predict(X_train)
ypred_test = mdl.predict(X_test)

print(classification_report(y_train, ypred_train))
print(classification_report(y_test, ypred_test))

              precision    recall  f1-score   support

           N       0.84      0.48      0.61       113
           Y       0.82      0.96      0.88       271

    accuracy                           0.82       384
   macro avg       0.83      0.72      0.75       384
weighted avg       0.82      0.82      0.80       384

              precision    recall  f1-score   support

           N       0.82      0.40      0.54        35
           Y       0.73      0.95      0.83        61

    accuracy                           0.75        96
   macro avg       0.78      0.68      0.68        96
weighted avg       0.77      0.75      0.72        96



In [ ]:
from sklearn.model_selection import GridSearchCV

param = {"C" : [0.001,0.01,0.1,0.5,1,10],
         "solver":["newton-cg", "lbfgs"],
         "class_weight":[{"N":1.5, "Y":1},{"N":1, "Y":1},{"N":2, "Y":1}]}

grid_search = GridSearchCV(LogisticRegression(), param, n_jobs = -1,cv = 5)
grid_search.fit(X_train, y_train)
grid_search.best_estimator_

LogisticRegression(C=0.1, class_weight={'N': 1.5, 'Y': 1}, solver='newton-cg')

In [ ]:
grid_search.cv_results_